In [1]:
# --- SAGEMAKER SETUP ---
# Import libraries Sagemaker (Container)

import boto3 # AWS SDK for Python. Used for direct S3 actions, clients, etc. (use it to upload CSVs) 
import os # filesystem / env var utilities.
import sagemaker # high-level SageMaker Python SDK. Provides Session, Estimator, helper classes and deployment tools
from sagemaker import get_execution_role # Convenience that returns the IAM role ARN when you run inside a SageMaker managed environment (Studio, notebook instance)
from sagemaker.serializers import CSVSerializer # Used by predictors to serialize inputs when calling a deployed endpoint (convert to text/csv)
from sagemaker.inputs import TrainingInput # Tells SageMaker where your training data lives (in S3) and how it should be loaded to .fit()

# Get the SageMaker session and the execution role from the SageMaker domain
sess = sagemaker.Session() # Creates an object that holds config (default S3 bucket, region) and helpers for uploading, describing jobs, etc.
role = get_execution_role() # An IAM role ARN that SageMaker uses to access S3, CloudWatch, ECR, etc. The role must have the right permissions (S3 write/read, SageMaker actions)

bucket = 'script-mode-xgb-demo' # The name of a bucket that is already created in S3
prefix = 'demo-xgb-hyperopt' # The name of the folder that will be created in the S3 bucket

print(f"Session bucket: {bucket}")
print(f"Prefix: {prefix}")
print("✅ SageMaker session ready")

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
Session bucket: script-mode-xgb-demo
Prefix: demo-xgb-hyperopt
✅ SageMaker session ready


In [2]:
# --- Read Training Dataset ---

import pandas as pd

# load dataset
df = pd.read_csv('data/training_data.csv')
df_copy = df.copy()
df_copy.head()

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,target
0,432.475954,289.373016,481.315600,358.755566,802.659004,176.761177,72.648102,720.969179,36.327684,83.768878,...,4.385848,516.789458,19.624422,13.162440,42.351948,35.920392,20.755984,13.814300,384.497136,14.364922
1,517.596250,330.448341,585.920055,22.684031,169.813240,335.601640,284.451476,748.101047,73.701438,358.147215,...,5.563334,2.960064,20.721878,17.740184,1.726915,167.576065,75.492679,2.480979,303.710869,19.984801
2,189.439350,553.888820,165.833790,202.465927,176.695586,321.155049,407.278389,161.245668,282.269025,221.570899,...,4.536947,581.823741,101.695639,0.653592,486.859084,117.491548,6.420465,20.713314,22.651537,12.944351
3,237.307878,195.894881,416.752252,468.729031,611.693517,301.411711,241.880655,49.597044,122.396821,13.828319,...,5.518968,45.014729,196.350455,47.638515,411.414213,67.142022,115.630943,8.927957,388.240433,14.792440
4,602.845256,16.103208,221.759979,345.765574,558.588369,276.704241,408.069566,19.390813,138.769765,146.662193,...,2.136214,133.590430,197.634584,26.278027,111.127557,172.181136,85.869642,30.537857,625.931837,11.802634


In [3]:
# --- Defining CSVs and target for Training ---

from sklearn.model_selection import train_test_split

X_full = df_copy.drop(columns='target')
y_full = df_copy['target']

# Test Split
# 80/20
X_temp, X_test, y_temp, y_test = train_test_split(X_full, y_full, test_size=0.2, random_state=12345)

# Validation Set (0.25 of 0.8 = 0.2)
X_train, X_valid, y_train, y_valid = train_test_split(X_temp, y_temp, test_size=0.25, random_state=12345)
# The test size is 25% of the training data (80%), which is 20% of the full dataset
# 60 training /20 validation / 20 test 

# --- Export to CSV files ---
train_df = pd.concat([X_train, y_train], axis=1)
valid_df = pd.concat([X_valid, y_valid], axis=1)
test_df  = pd.concat([X_test, y_test], axis=1)

train_df.to_csv("train.csv", index=False)
valid_df.to_csv("validation.csv", index=False)
test_df.to_csv("test.csv", index=False)

print("✅ Local CSVs created successfully:")
print(f" - train.csv ({train_df.shape})")
print(f" - validation.csv ({valid_df.shape})")
print(f" - test.csv ({test_df.shape})")

✅ Local CSVs created successfully:
 - train.csv ((480, 21))
 - validation.csv ((160, 21))
 - test.csv ((160, 21))


In [4]:
# --- Upload training, testing, and validation data to the S3 bucket ---
# SageMaker can only access data stored in S3

# Path used: s3://<bucket>/<prefix>/train/train.csv, etc.
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'validation/validation.csv')).upload_file('validation.csv')
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'test/test.csv')).upload_file('test.csv')

# Define the S3 inputs for SageMaker
# content_type is the input parsing: how the training container reads the input files once it downloads them from S3
s3_input_train = TrainingInput(
    s3_data='s3://{}/{}/train'.format(bucket, prefix), content_type='csv'
)
s3_input_validation = TrainingInput(
    s3_data='s3://{}/{}/validation/'.format(bucket, prefix), content_type='csv'
)
s3_input_test = TrainingInput(
    s3_data='s3://{}/{}/test/'.format(bucket, prefix), content_type='csv'
)

print(f"✅ Train data uploaded to: {s3_input_train.config['DataSource']['S3DataSource']['S3Uri']}")
print(f"✅ Validation data uploaded to: {s3_input_validation.config['DataSource']['S3DataSource']['S3Uri']}")
print(f"✅ Test data uploaded to: {s3_input_test.config['DataSource']['S3DataSource']['S3Uri']}")

In [5]:
# --- TRAINING THE MODEL ---

# --- Create XGBoost Estimator ---
from sagemaker.estimator import Estimator

# How to run the training job: which container, script, instance, and dependencies
xgb_estimator = Estimator(
    image_uri='683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:1.5-1',  # XGBoost container
    entry_point='train.py',          # The custom script
    source_dir='.',                  # Uploads train.py, preprocessing.py, etc.
    role=role,                       # IAM role for SageMaker
    instance_count=1,                # One instance
    instance_type='ml.m5.xlarge',    # Compute type 4 vCPUs, 16GB RAM
    base_job_name='xgb-train-demo',  # Job name prefix
    dependencies=['requirements.txt'] # An specific version of Pandas
)

# --- Launch training job ---
print("🚀 Launching XGBoost training job...")

# 1. SageMaker spins up a training instance (like ml.m5.xlarge).
# 2. Downloads your container image (in this case, the XGBoost image from ECR).
# 3. Uploads your scripts (train.py, requirements.txt, etc.) to the container.
# 4. Downloads your data from the S3 URIs you passed in (s3_input_train, etc.).
# 5. Runs your script inside the container

# Execution of the Training job
xgb_estimator.fit({
    'train': s3_input_train,
    'validation': s3_input_validation,
    'test': s3_input_test
})

print("✅ XGBoost training job launched successfully!")

🚀 Launching XGBoost training job...


INFO:sagemaker:Creating training-job with name: xgb-train-demo-2025-10-21-20-32-50-274


2025-10-21 20:32:52 Starting - Starting the training job...
2025-10-21 20:33:23 Downloading - Downloading input data...
2025-10-21 20:33:44 Downloading - Downloading the training image......
2025-10-21 20:34:35 Training - Training image download completed. Training in progress./miniconda3/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
[2025-10-21 20:34:42.887 ip-10-0-240-154.ec2.internal:7 INFO utils.py:28] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2025-10-21 20:34:42.910 ip-10-0-240-154.ec2.internal:7 INFO profiler_config_parser.py:111] User has disabled profiler.
[2025-10-21:20:34:43:INFO] Imported framework sagemaker_xgboost_container.training
[2025-10-21:20:34:43:INFO] No GPUs detected (normal if no gpus installed)
[2025-10-21:20:34:43:INFO] Invoking user training script.
[2025-10-21:20:34:

In [10]:
# --- Where the model is (S3 SageMaker bucket) ---

import sagemaker

sess = sagemaker.Session()
job_name = "xgb-train-demo-2025-10-21-20-32-50-274"  # last job name
desc = sess.describe_training_job(job_name)
print("✅ Training job status:", desc["TrainingJobStatus"])
print("📦 Model S3 Path:", desc["ModelArtifacts"]["S3ModelArtifacts"])

✅ Training job status: Completed
📦 Model S3 Path: s3://sagemaker-us-east-1-059535450142/xgb-train-demo-2025-10-21-20-32-50-274/output/model.tar.gz


In [11]:
# --- What was the container where the model was trained ---

sagemaker_session = sagemaker.session.Session()
job_name = 'xgb-train-demo-2025-10-21-20-32-50-274' # last job name

training_job = sagemaker_session.sagemaker_client.describe_training_job(TrainingJobName=job_name)
print("📦 The Model Container is:",training_job['AlgorithmSpecification']['TrainingImage'])

📦 The Model Container is: 683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:1.5-1


In [16]:
# --- DEPLOYMENT AND INFERENCE WITH AN ENDPOINT ---

import os # Allow the interaction with the operating system: creating folders, reading env variables
import boto3 # AWS SDK for Python. Used for direct S3 actions, clients, etc. (use it to upload CSVs) 
import pandas as pd
from preprocessing import preprocess_data # Using the preprocessing script for feature engineering 
import sagemaker # High-level SageMaker Python SDK. Provides Session, Estimator, helper classes and deployment tools
from sagemaker import Session # Session represents the connection to the SageMaker environment

# Load the new dataset
df = pd.read_csv("data/blind_test_data.csv")
df_processed = preprocess_data(df, is_training=False)

# Save to a local CSV file for upload
local_file = "inference_data.csv"
df_processed.to_csv(local_file, header=False, index=False)
print("✅ The dataset for prediction is ready:")
print(df_processed)

# -------------------------------------
# Upload the dataset to the specific S3 Bucket
bucket_inference = 'script-mode-xgb-demo'  # The existing S3 bucket
prefix_inference = 'demo-xgb-hyperopt'    # Folder in the bucket
s3_key = os.path.join(prefix_inference, 'xgb_data_input', 'inference_data.csv')

s3 = boto3.client('s3') # Creates an S3 client using the boto3 library (AWS SDK for Python)
s3.upload_file(local_file, bucket_inference, s3_key) # Uploads the local file (local_file = "inference_data.csv") to S3
print(f"✅ Dataset uploaded to S3 at: s3://{bucket_inference}/{s3_key}")
# ---------------------------------------

# Deploy the model to a new endpoint, which creates the endpoint configuration and the endpoint
xgb_predictor = xgb_estimator.deploy(
    initial_instance_count=1,
    instance_type='ml.m4.xlarge'
)
print(f"✅ Model deployed successfully. Endpoint name: {xgb_predictor.endpoint_name}")

# Create a low-level client for the SageMaker Runtime 
sagemaker_runtime = boto3.client('sagemaker-runtime') # Sends requests to deployed endpoints and get real time predictions (inference) back

# Run inference
print("⚡ Running inference...")
# Open the file file.txt for reading, call it f, and automatically close it when I’m done — even if something goes wrong
with open(local_file, "r") as f: # Opens the local CSV file (inference_data.csv) that contains the new data you want predictions for
    payload = f.read() # Reads all its contents into a string variable called payload. Prepares the CSV as a raw text body ready to be sent over the network.

# Call the endpoint to get predictions
response = sagemaker_runtime.invoke_endpoint(
    EndpointName=xgb_predictor.endpoint_name,
    Body=payload, # The actual data to send for prediction (the CSV)
    ContentType='text/csv' # Tells SageMaker how to interpret the payload (CSV format)
)

# --- Print predictions ---
predictions = response['Body'].read().decode('utf-8')
print("✅ Predictions received (resume):")
print(predictions[:220])  # Preview of the 220 characters of the variable predictions

# --- Save predictions to CSV file ---
output_name = "predictions_from_endpoint.csv"
with open(output_name, "w") as f: # Write 
    f.write(predictions)
print(f"📁 Predictions saved locally as: {output_name}")

# -------------------------------------
# Upload the predictions file to S3
s3_output_key = os.path.join(prefix_inference, 'xgb_prediction_output', 'predictions_from_endpoint.csv')
s3.upload_file(output_name, bucket_inference, s3_output_key)
print(f"✅ Predictions uploaded to S3 at: s3://{bucket_inference}/{s3_output_key}")
# -------------------------------------

# --- Clean up the endpoint ---
print("🧹 Cleaning up endpoint to avoid extra charges...")
try:
    xgb_predictor.delete_endpoint()
    print(f"✅ Endpoint '{xgb_predictor.endpoint_name}' deleted successfully.")
except Exception as e:
    print(f"⚠️ Could not delete endpoint: {e}")

print("---All steps completed successfully!---")

INFO:sagemaker:Creating model with name: xgb-train-demo-2025-10-21-22-29-32-534


✅ The dataset for prediction is ready:
     feature_2_9_13  feature_9_x_13  feature_11  feature_18   feature_2
0      1.171644e+07    45978.223946    0.237996   26.969870  254.825875
1      9.574306e+05     2758.607361    6.608084   33.500538  347.070280
2      1.297937e+07    52890.709578    1.822370    6.211169  245.399775
3      3.305780e+06    46902.366963    1.686895   15.001753   70.482159
4      5.944353e+06    12183.850025    0.948507    3.443809  487.887860
..              ...             ...         ...         ...         ...
195    1.570685e+06    19167.693374    5.948526   19.159594   81.944368
196    3.671356e+06    33810.532103    1.021152   27.156436  108.586165
197    8.704712e+05     1986.777393    1.438832   20.496401  438.132233
198    5.049940e+06    12209.519713    0.106124   50.045511  413.606784
199    6.073951e+06    34500.992248    5.850888   20.758504  176.051484

[200 rows x 5 columns]
✅ Dataset uploaded to S3 at: s3://script-mode-xgb-demo/demo-xgb-hyperopt/

INFO:sagemaker:Creating endpoint-config with name xgb-train-demo-2025-10-21-22-29-32-534
INFO:sagemaker:Creating endpoint with name xgb-train-demo-2025-10-21-22-29-32-534


-------!✅ Model deployed successfully. Endpoint name: xgb-train-demo-2025-10-21-22-29-32-534
⚡ Running inference...
✅ Predictions received (resume):
14.245513916015625
13.315871238708496
16.2331485748291
13.069272994995117
16.720584869384766
10.739200592041016
19.85389518737793
18.805179595947266
12.95263957977295
8.774255752563477
18.93240737915039
22.75801658630371
📁 Predictions saved locally as: predictions_from_endpoint.csv


INFO:sagemaker:Deleting endpoint configuration with name: xgb-train-demo-2025-10-21-22-29-32-534


✅ Predictions uploaded to S3 at: s3://script-mode-xgb-demo/demo-xgb-hyperopt/xgb_prediction_output/predictions_from_endpoint.csv
🧹 Cleaning up endpoint to avoid extra charges...


INFO:sagemaker:Deleting endpoint with name: xgb-train-demo-2025-10-21-22-29-32-534


✅ Endpoint 'xgb-train-demo-2025-10-21-22-29-32-534' deleted successfully.
🎯 All steps completed successfully!
